In [3]:
import os
from shutil import copy, move
import pandas as pd
import numpy as np

In [1]:
from roboflow import Roboflow

rf = Roboflow(api_key='')
project = rf.workspace("tid-detection").project("nasatiddetect")
dataset = project.version(11).download("yolov7")
!ls

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to NasaTIDDetect-11 in yolov7pytorch:: 100%|█| 2


10_14_pull_height.csv
11-1-22_Relevant.ipynb
2017_gray_img_plots_set_A_cropped
2017_gray_img_plots_set_B_cropped
2017_img_csvs
2017_img_plots
2017_img_plots_set_A
2017_img_plots_set_A_cropped
2017_img_plots_set_B
2017_img_plots_set_B_cropped
2017_img_plots_zipped
CNNForTabularData.pdf
CompareHamBeaconTIDs.xlsx
DataHandling
DatabaseRetrieve
ImgData.xlsx
NasaTIDDetect-11
NasaTIDDetect-8.ipynb
NasaTIDDetect-8_xml
Notebooks
ObjectDetectionmAP.PNG
Papers
TID_set_Lucas.csv
TID_set_Nick.csv
TIDs_DataFrame.csv
YoloHandling.ipynb
breast_ultrasound.zip
ionosphere_detections
sample_plot_csv.csv
training_labeled
wa9vnj_eclipse_smooth.ipynb
wsprspots-2017-01.csv


In [4]:
os.chdir('NasaTIDDetect-11')
!ls

README.dataset.txt
README.roboflow.txt
data.yaml
test
train
valid


In [ ]:
!pip install kaggle

In [ ]:
P_DIR = 'test/'

def get_paths(pdir, ext='')
    if ext:
        file_paths = [os.path.join(pdir, file) for file in os.listdir(pdir) if file.endswith(ext)]
    else:
        file_paths = [os.path.join(pdir, file) for file in os.listdir(pdir)]
    return

jpg_paths = get_paths(os.path.join(P_DIR , 'images/'), '.jpg')
txt_paths = get_paths(os.path.join(P_DIR , 'labels/'), '.txt')

def convert_to_bboxes(file_paths, centered=True):
    classes = list()
    for file in file_paths:
        with open(file, 'r'), open(file.replace('.txt','_bbox.txt'), 'w') as f, o:
            for line in f.readline():
                class_, points = line.split(' ', 1)
                class_ = int(class_)
                if class_ not in classes: classes.append(class_)
                points = [(points[i], points[i+1]) for i in range(0,len(points),2)]
                xs, ys = zip(*points)
                xmin, xmax, ymin, ymax = min(xs), min(ys), max(xs), max(ys)
                if centered:
                    coords = (xmin + xmax) / 2, (ymin + ymax) / 2, xmax - xmin, ymax-ymin
                else:
                    coords = xmin, ymin, xmax, ymax
                o.write(f'{class_} {coords[0]} {coords[1]} {coords[2]} {coords[3]}\n')

[YOLOv7 Instance](https://stackabuse.com/instance-segmentation-with-yolov7-in-python/)

In [ ]:
%cd yolov7
! curl -L https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-mask.pt -o yolov7-mask.pt
# hyp.scratch.mask.yaml

In [ ]:
with open('data/hyp.scratch.mask.yaml') as f:
    hyp = yaml.load(f, Loader=yaml.FullLoader)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def load_model():
    model = torch.load('yolov7-mask.pt', map_location=device)['model']
    # Put in inference mode
    model.eval()

    if torch.cuda.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)
    return model

model = load_model()

In [ ]:
def run_inference(url):
    image = cv2.imread(url) # shape: (480, 640, 3)
    # Resize and pad image
    image = letterbox(image, 640, stride=64, auto=True)[0] # shape: (480, 640, 3)
    # Apply transforms
    image = transforms.ToTensor()(image) # torch.Size([3, 480, 640])
    # Match tensor type (`torch.FloatTensor` -> `torch.HalfTensor`) with model
    image = image.half().to(device)
    # Turn image into batch
    image = image.unsqueeze(0) # torch.Size([1, 3, 480, 640])
    output = model(image)
    return output, image

output, image = run_inference('../street.png')

[YOLOv7 Instance Custom Data](https://medium.com/augmented-startups/train-yolov7-segmentation-on-custom-data-b91237bd2a29)


In [ ]:
!git clone https://github.com/RizwanMunawar/yolov7-segmentation.git
!cd yolov7-segmentation
!pip install --upgrade pip

#for linux users
!pip install -r requirements.txt
!sudo apt install -y zip htop screen libgl1-mesa-glx

In [ ]:
!python train.py --data data/custom.yaml --batch 4 --weights '' --cfg yolov7-seg.yaml --epochs 10 --name yolov7-seg --img 640 --hyp hyp.scratch-high.yaml

In [20]:
df = pd.read_csv('all_labels_v1.csv')
df['filename'] = df['filename'].apply(lambda x : x.replace('_v1',''))
df['class'] = 'TIDs'
display(df)
df_list = list()
for filename in df['filename'].unique():
    df_list.append(df.loc[df['filename'] == filename,:].iloc[0,:])
    
df = pd.concat(df_list, axis=1).transpose().set_index('filename')
display(df)

df['xmin_n'] = df['xmin'] / df['width']
df['ymin_n'] = df['ymin'] / df['height']
df['xmax_n'] = df['xmax'] / df['width']
df['ymax_n'] = df['ymax'] / df['height']

df['xmin_n'] = df['xmin'] / df['width']
df['ymin_n'] = df['ymin'] / df['height']
df['xmax_n'] = df['xmax'] / df['width']
df['ymax_n'] = df['ymax'] / df['height']

df['xc'] = (df['xmax_n'] + df['xmin_n']) / 2
df['yc'] = (df['ymax_n'] + df['ymin_n']) / 2
df['dx'] = df['xmax_n'] - df['xmin_n']
df['dy'] = df['ymax_n'] - df['ymin_n']
df.to_csv('2017_ALL_INFO.csv')
df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,2017-01-02.jpg,719,301,TIDs,263,105,653,208
1,2017-01-02.jpg,719,301,TIDs,263,105,369,208
2,2017-01-02.jpg,719,301,TIDs,263,105,440,208
3,2017-01-02.jpg,719,301,TIDs,263,105,511,208
4,2017-01-02.jpg,719,301,TIDs,263,105,582,208
...,...,...,...,...,...,...,...,...
3133,2017-12-24.jpg,719,301,TIDs,189,132,556,243
3134,2017-12-24.jpg,719,301,TIDs,185,132,556,243
3135,2017-12-24.jpg,719,301,TIDs,159,118,556,256
3136,2017-12-24.jpg,719,301,TIDs,159,104,556,270


,width,height,class,xmin,ymin,xmax,ymax
filename,,,,,,,
2017-01-02.jpg,719,301,TIDs,263,105,653,208
2017-01-03.jpg,719,301,TIDs,157,99,670,219
2017-01-05.jpg,719,301,TIDs,479,102,717,199
2017-01-06.jpg,719,301,TIDs,182,63,604,215
2017-01-07.jpg,719,301,TIDs,237,107,637,245
...,...,...,...,...,...,...,...
2017-12-23.jpg,719,301,TIDs,183,73,470,222
2017-12-24.jpg,719,301,TIDs,159,132,556,243
2017-12-25.jpg,719,301,TIDs,120,43,718,227


,width,height,class,xmin,ymin,xmax,ymax,xmin_n,ymin_n,xmax_n,ymax_n,xc,yc,dx,dy
filename,,,,,,,,,,,,,,,
2017-01-02.jpg,719,301,TIDs,263,105,653,208,0.365786,0.348837,0.908206,0.69103,0.636996,0.519934,0.54242,0.342193
2017-01-03.jpg,719,301,TIDs,157,99,670,219,0.218359,0.328904,0.93185,0.727575,0.575104,0.528239,0.713491,0.398671
2017-01-05.jpg,719,301,TIDs,479,102,717,199,0.666203,0.33887,0.997218,0.66113,0.831711,0.5,0.331015,0.322259
2017-01-06.jpg,719,301,TIDs,182,63,604,215,0.253129,0.209302,0.840056,0.714286,0.546592,0.461794,0.586926,0.504983
2017-01-07.jpg,719,301,TIDs,237,107,637,245,0.329624,0.355482,0.885953,0.813953,0.607789,0.584718,0.556328,0.458472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-23.jpg,719,301,TIDs,183,73,470,222,0.25452,0.242525,0.653686,0.737542,0.454103,0.490033,0.399166,0.495017
2017-12-24.jpg,719,301,TIDs,159,132,556,243,0.22114,0.438538,0.773296,0.807309,0.497218,0.622924,0.552156,0.368771
2017-12-25.jpg,719,301,TIDs,120,43,718,227,0.166898,0.142857,0.998609,0.754153,0.582754,0.448505,0.831711,0.611296


In [24]:
df.loc[:,['xmin_n','xmax_n']].applymap(lambda x : round(x, 2)).to_csv('11-5-Labeling.csv')

In [41]:
img_data = pd.read_excel('ImgData.xlsx', index_col=0)
img_data = img_data.loc[:,['Start time','End time']].dropna()
img_data = img_data.where(img_data != ' ').dropna()
img_data = img_data.astype(np.int64) - 12
img_data = img_data / 12
img_data.columns = ['xmin','xmax']
img_data.index.name = 'Date'
img_data = img_data.sort_index()
img_data.index = ['d' + date.strftime('%m-%d-%Y') for date in img_data.index]
# display(img_data)
img_data.applymap(lambda x : round(x, 2)).to_csv('11-5-Labeling.csv')

In [ ]:
# https://universe.roboflow.com/mohamed-traore-2ekkp/chest-x-rays-qjmia/dataset/4
# https://www.kaggle.com/competitions/vinbigdata-chest-xray-abnormalities-detection/data